In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate


2023-10-24 17:05:54.454914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Load the augmented data from the saved file
# data = np.load('/content/drive/My Drive/Capstone/Pickled/coral_augmented_combined_RCNN.npz')

# # Access the arrays from the loaded data
# X_train = data['X_train']
# y_train = data['y_train']
# X_test = data['X_test']
# y_test = data['y_test']

# # Check the shape of the loaded data
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
X_train shape: (5104, 224, 224, 3)
y_train shape: (5104, 224, 224)
X_test shape: (1276, 224, 224, 3)
y_test shape: (1276, 224, 224)


In [2]:
# Load the augmented data from the saved file
data = np.load('../data/external/coral_augmented_combined_RCNN.npz')

# Access the arrays from the loaded data
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

# Check the shape of the loaded data
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (5104, 224, 224, 3)
y_train shape: (5104, 224, 224)
X_test shape: (1276, 224, 224, 3)
y_test shape: (1276, 224, 224)


In [3]:
def simplified_unet(input_shape=(224, 224, 3)):
    inputs = Input(input_shape)

    # Contracting Path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    bottleneck = Conv2D(256, 3, activation='relu', padding='same')(pool2)

    # Expansive Path
    up2 = Concatenate()([UpSampling2D(size=(2, 2))(bottleneck), conv2])
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(up2)

    up1 = Concatenate()([UpSampling2D(size=(2, 2))(conv3), conv1])
    conv4 = Conv2D(64, 3, activation='relu', padding='same')(up1)

    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv4)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [4]:
# Create the U-Net model
model = simplified_unet()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


2023-10-24 17:06:28.196715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
160/160 [==============================] - 2483s 16s/step - loss: 4.5503e-06 - accuracy: 1.0000 - val_loss: 9.1482e-12 - val_accuracy: 1.0000
Epoch 2/10
160/160 [==============================] - 4063s 25s/step - loss: 4.8321e-11 - accuracy: 1.0000 - val_loss: 9.1296e-12 - val_accuracy: 1.0000
Epoch 3/10
160/160 [==============================] - ETA: 0s - loss: 4.8170e-11 - accuracy: 1.0000 

KeyboardInterrupt: 

In [ ]:
model.save('../models/coral_U-net.h5')